In [3]:
import pandas as pd

emotion_train = pd.read_csv('train.txt', sep=';', header=None, names=['text', 'emotion'])
emotion_val = pd.read_csv('val.txt', sep=';', header=None, names=['text', 'emotion'])
emotion_test = pd.read_csv('test.txt', sep=';', header=None, names=['text', 'emotion'])

print("Emotion Train Sample:")
print(emotion_train.head())
print("\nEmotion Validation Sample:")
print(emotion_val.head())
print("\nEmotion Test Sample:")
print(emotion_test.head())


Emotion Train Sample:
                                                text  emotion
0                            i didnt feel humiliated  sadness
1  i can go from feeling so hopeless to so damned...  sadness
2   im grabbing a minute to post i feel greedy wrong    anger
3  i am ever feeling nostalgic about the fireplac...     love
4                               i am feeling grouchy    anger

Emotion Validation Sample:
                                                text  emotion
0  im feeling quite sad and sorry for myself but ...  sadness
1  i feel like i am still looking at a blank canv...  sadness
2                     i feel like a faithful servant     love
3                  i am just feeling cranky and blue    anger
4  i can have for a treat or if i am feeling festive      joy

Emotion Test Sample:
                                                text  emotion
0  im feeling rather rotten so im not very ambiti...  sadness
1          im updating my blog because i feel shitty  sadnes

In [4]:
movies = pd.read_csv('movies_metadata.csv')
ratings = pd.read_csv('ratings.csv')

credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')
links = pd.read_csv('links.csv')

ratings_small = pd.read_csv('ratings_small.csv')
links_small = pd.read_csv('links_small.csv')

print("Movies Sample:")
print(movies.head())
print("\nRatings Sample:")
print(ratings.head())
print("\nCredits Sample:")
print(credits.head())
print("\nKeywords Sample:")
print(keywords.head())
print("\nLinks Sample:")
print(links.head())


C:\Users\Sendhurvelan.A.P\AppData\Local\Temp\ipykernel_16280\3551232640.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('movies_metadata.csv')


Movies Sample:
   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0

In [5]:
print(movies.shape, movies.columns)
print(ratings.shape, ratings.columns)

(45466, 24) Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
(26024289, 4) Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')


In [6]:
import nltk
from nltk.corpus import stopwords
import string


nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
  
    text = text.lower()
  
    text = text.translate(str.maketrans('', '', string.punctuation))
   
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)


emotion_train['clean_text'] = emotion_train['text'].apply(clean_text)
emotion_val['clean_text'] = emotion_val['text'].apply(clean_text)
emotion_test['clean_text'] = emotion_test['text'].apply(clean_text)


print(emotion_train[['text', 'clean_text', 'emotion']].head())


[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


                                                text  \
0                            i didnt feel humiliated   
1  i can go from feeling so hopeless to so damned...   
2   im grabbing a minute to post i feel greedy wrong   
3  i am ever feeling nostalgic about the fireplac...   
4                               i am feeling grouchy   

                                          clean_text  emotion  
0                              didnt feel humiliated  sadness  
1  go feeling hopeless damned hopeful around some...  sadness  
2          im grabbing minute post feel greedy wrong    anger  
3  ever feeling nostalgic fireplace know still pr...     love  
4                                    feeling grouchy    anger  


In [7]:
print(emotion_train.columns)

Index(['text', 'emotion', 'clean_text'], dtype='object')


In [8]:
print(emotion_train.columns.tolist())
print(emotion_train.head())


['text', 'emotion', 'clean_text']
                                                text  emotion  \
0                            i didnt feel humiliated  sadness   
1  i can go from feeling so hopeless to so damned...  sadness   
2   im grabbing a minute to post i feel greedy wrong    anger   
3  i am ever feeling nostalgic about the fireplac...     love   
4                               i am feeling grouchy    anger   

                                          clean_text  
0                              didnt feel humiliated  
1  go feeling hopeless damned hopeful around some...  
2          im grabbing minute post feel greedy wrong  
3  ever feeling nostalgic fireplace know still pr...  
4                                    feeling grouchy  


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
def get_clean_text(df):
   
    for col in ['clean_text', 'text_clean', 'text', 'cleaned_text']:
        if col in df.columns:
            return df[col]
    raise ValueError("No suitable text column found in dataframe.")

X_train = vectorizer.fit_transform(get_clean_text(emotion_train))
X_val = vectorizer.transform(get_clean_text(emotion_val))
X_test = vectorizer.transform(get_clean_text(emotion_test))


def get_label(df, label_col='emotion'):
    if label_col in df.columns:
        return df[label_col]
    raise ValueError(f"Label column '{label_col}' not found in dataframe.")

y_train = get_label(emotion_train)
y_val = get_label(emotion_val)
y_test = get_label(emotion_test)

print("Train feature matrix shape:", X_train.shape)
print("Validation feature matrix shape:", X_val.shape)
print("Test feature matrix shape:", X_test.shape)


Train feature matrix shape: (16000, 5000)
Validation feature matrix shape: (2000, 5000)
Test feature matrix shape: (2000, 5000)


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))


              precision    recall  f1-score   support

       anger       0.90      0.84      0.87       275
        fear       0.85      0.77      0.81       212
         joy       0.87      0.95      0.91       704
        love       0.89      0.73      0.80       178
     sadness       0.88      0.94      0.91       550
    surprise       0.86      0.62      0.72        81

    accuracy                           0.88      2000
   macro avg       0.88      0.81      0.84      2000
weighted avg       0.88      0.88      0.87      2000



In [11]:
import pandas as pd

movies = pd.read_csv('movies_metadata.csv', low_memory=False)
ratings = pd.read_csv('ratings.csv')


print(movies[['id', 'title', 'genres']].head())
print(ratings.head())


      id                        title  \
0    862                    Toy Story   
1   8844                      Jumanji   
2  15602             Grumpier Old Men   
3  31357            Waiting to Exhale   
4  11862  Father of the Bride Part II   

                                              genres  
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...  
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...  
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...  
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...  
4                     [{'id': 35, 'name': 'Comedy'}]  
   userId  movieId  rating   timestamp
0       1      110     1.0  1425941529
1       1      147     4.5  1425942435
2       1      858     5.0  1425941523
3       1     1221     5.0  1425941546
4       1     1246     5.0  1425941556


In [12]:
movies = movies[movies['id'].apply(lambda x: x.isnumeric())]
movies['id'] = movies['id'].astype(int)
ratings['movieId'] = ratings['movieId'].astype(int)
movies = movies.rename(columns={'id': 'movieId'})
movie_ratings = pd.merge(ratings, movies, on='movieId')
print(movie_ratings[['userId', 'movieId', 'rating', 'title', 'genres']].head())


   userId  movieId  rating                 title  \
0       1      110     1.0     Three Colors: Red   
1       1      147     4.5         The 400 Blows   
2       1      858     5.0  Sleepless in Seattle   
3       1     1246     5.0          Rocky Balboa   
4       1     1968     4.0         Fools Rush In   

                                              genres  
0  [{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...  
1                      [{'id': 18, 'name': 'Drama'}]  
2  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...  
3                      [{'id': 18, 'name': 'Drama'}]  
4  [{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...  


In [13]:
top_users = ratings['userId'].value_counts().head(1000).index
ratings_small_users = ratings[ratings['userId'].isin(top_users)]

top_movies = ratings['movieId'].value_counts().head(1000).index
ratings_small = ratings_small_users[ratings_small_users['movieId'].isin(top_movies)]

user_item_matrix_small = ratings_small.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

print(user_item_matrix_small.shape)


(1000, 1000)


In [14]:
top_users = ratings['userId'].value_counts().head(1000).index
ratings_small_users = ratings[ratings['userId'].isin(top_users)]
top_movies = ratings_small_users['movieId'].value_counts().head(1000).index
ratings_small = ratings_small_users[ratings_small_users['movieId'].isin(top_movies)]

print("Filtered ratings shape:", ratings_small.shape)

user_item_matrix_small = ratings_small.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

print("User-item matrix shape:", user_item_matrix_small.shape)
print(user_item_matrix_small.head())


Filtered ratings shape: (651837, 4)
User-item matrix shape: (1000, 1000)
movieId  1       2       6       10      11      16      19      21      \
userId                                                                    
741         4.5     3.0     3.5     3.0     0.0     0.0     3.0     4.0   
836         3.5     3.5     0.0     0.0     0.0     0.0     1.5     0.0   
1846        3.5     3.0     5.0     3.5     0.0     3.5     2.5     0.0   
1932        3.5     4.0     4.0     3.0     0.0     0.0     3.5     2.5   
2520        0.0     0.0     0.0     0.0     0.0     4.5     0.0     0.0   

movieId  22      25      ...  89745   91500   91529   94864   96079   96610   \
userId                   ...                                                   
741         0.0     0.0  ...     3.5     0.0     3.5     4.0     4.0     3.5   
836         0.0     0.0  ...     4.0     3.5     4.0     4.0     4.0     4.5   
1846        0.0     0.0  ...     3.5     4.0     4.0     3.0     4.5     4.0   
1

In [15]:
print(user_item_matrix_small.index)


Index([   741,    836,   1846,   1932,   2520,   2531,   2547,   2975,   3140,
         3437,
       ...
       267799, 268146, 268239, 268356, 269034, 270071, 270123, 270213, 270654,
       270887],
      dtype='int64', name='userId', length=1000)


In [16]:
def recommend_movies_small(user_id, user_item_matrix, user_similarity_df, n_recommendations=5):
    user_ratings = user_item_matrix.loc[user_id]
    unrated_movies = user_ratings[user_ratings == 0].index
    movie_scores = user_similarity_df.loc[user_id][unrated_movies]
    top_n = movie_scores.sort_values(ascending=False).head(n_recommendations)
    recommendations = top_n.index
    return recommendations
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
user_similarity = cosine_similarity(user_item_matrix_small)
user_similarity_df_small = pd.DataFrame(user_similarity, 
    index=user_item_matrix_small.index, 
    columns=user_item_matrix_small.index)


In [17]:
def recommend_movies_small(user_id, user_item_matrix, user_similarity_df, n_recommendations=5):
    
    user_ratings = user_item_matrix.loc[user_id]
    
    similarity_scores = user_similarity_df.loc[user_id]
    
    similarity_scores = similarity_scores.drop(user_id)
    
    unrated_movies = user_ratings[user_ratings == 0].index
    weighted_scores = {}
    
    for movie in unrated_movies:

        other_ratings = user_item_matrix[movie]

        relevant_ratings = other_ratings[similarity_scores.index]
        weights = similarity_scores.loc[relevant_ratings.index]
        
        weighted_score = (relevant_ratings * weights).sum() / weights.sum()
        weighted_scores[movie] = weighted_score
    
    sorted_movies = sorted(weighted_scores.items(), key=lambda x: x[1], reverse=True)
    

    recommended_movies = pd.DataFrame(sorted_movies, columns=['movieId', 'score']).head(n_recommendations)
    
    return recommended_movies


In [42]:
import random

mood_to_genres = {
    'happy': ['Comedy', 'Family', 'Animation'],
    'sad': ['Drama', 'Biographies'],
    'angry': ['Action', 'Thriller', 'Crime'],
    'love': ['Romance', 'Musical'],
    'joy': ['Comedy', 'Adventure'],
    'fear': ['Horror', 'Mystery'],
    'surprise': ['Mystery', 'Fantasy'],
    'disgust': ['Thriller', 'Documentary'],
    'anger': ['Action', 'War'],
}

mood_to_emoji = {
    'happy': '😊',
    'sad': '😢',
    'angry': '😡',
    'love': '❤️',
    'joy': '😂',
    'fear': '😱',
    'surprise': '😲',
    'disgust': '🤢',
    'anger': '🔥',
}

fun_facts = {
    'happy': ["Did you know? Happy movies can boost your mood! 😊", "Smile! Watching comedy releases feel-good hormones."],
    'sad': ["It's okay to feel blue sometimes. Sad movies can be cathartic. 💧", "Feeling sentimental? Drama movies often hit the right notes."],
    'angry': ["Channel that anger into action movies! 🚀", "Break the tension with some adrenaline-pumping action.🔥"],
    'love': ["Love stories warm the heart and soothe the soul. 💖", "Romantic films bring us closer to our feelings."],
    'joy': ["Joyful movies lift spirits high! 🌟", "Laugh out loud with these happy picks."],
    'fear': ["Scary movies offer a safe thrill. 😱", "Pump up the adrenaline with horrors and thrillers."],
    'surprise': ["Unexpected twists keep the mind sharp! 🤯", "Surprise movies are filled with unexpected fun."],
    'disgust': ["Dark and gritty tales to challenge your senses. 🧟", "Sometimes the unsettling storylines engage deeply."],
    'anger': ["Fiery and intense movies for the fiery mood. 🔥", "Sometimes action is the best outlet for anger."]
}

messages = {
    'happy': "Keep smiling with these joyful picks! 🎉",
    'sad': "Sometimes, a good drama helps heal hearts. 💙",
    'angry': "Time to blow off steam with intense movies! 💥",
    'love': "Feel the heart flutter with these romantic gems. 💘",
    'joy': "Laugh and have fun with these joyous flicks! 😄",
    'fear': "Get your adrenaline rush with these chilling thrillers. 👻",
    'surprise': "Expect the unexpected with these surprise hits! 🎢",
    'disgust': "Brace yourself for some dark and gripping stories. 🕷️",
    'anger': "Channel your fury into these high-action movies! 🔥",
}

def recommend_unique_mood_movies(mood, movies_df, n_recommendations=5):
    mood = mood.lower()
    genres = mood_to_genres.get(mood)
    emoji = mood_to_emoji.get(mood, '🎬')
    fun_fact = random.choice(fun_facts.get(mood, ["Enjoy your movie!"]))
    message = messages.get(mood, "Enjoy your movie night!")

    if not genres:
        print(f"\nNo recommendations: '{mood}' is not a valid mood.")
        print(f"Please enter one of these moods: {', '.join(mood_to_genres.keys())}\n")
        return

    print(f"\n{fun_fact}\n")
    print(f"{message}\n")

    mask = movies_df['genres'].apply(lambda x: any(g in x for g in genres))
    filtered_movies = movies_df[mask]
    recommended = filtered_movies.head(n_recommendations)

    if recommended.empty:
        print(f"No movies found for '{mood}' mood and genres {genres}.")
        return

    print(f"**Here are your {mood.capitalize()} mood recommendations:** {emoji}\n")
    for i, row in recommended.iterrows():
        title = row['title']
        genres = row['genres']
        print(f"⭐ {title} ({genres}) {emoji}")

user_mood = input("Enter your mood (e.g. happy, sad, angry, love, fear, joy, surprise, disgust): ")
recommend_unique_mood_movies(user_mood, movies, 5)

Enter your mood (e.g. happy, sad, angry, love, fear, joy, surprise, disgust):  sad



It's okay to feel blue sometimes. Sad movies can be cathartic. 💧

Sometimes, a good drama helps heal hearts. 💙

**Here are your Sad mood recommendations:** 😢

⭐ Waiting to Exhale ([{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]) 😢
⭐ Heat ([{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}, {'id': 53, 'name': 'Thriller'}]) 😢
⭐ Tom and Huck ([{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}]) 😢
⭐ The American President ([{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]) 😢
⭐ Nixon ([{'id': 36, 'name': 'History'}, {'id': 18, 'name': 'Drama'}]) 😢
